### Automating Model Runs - Streamflow Capture Analysis

All groundwater pumped is balanced by removal of water somewhere, initially from storage in the aquifer and later from capture in the form of increase in recharge and decrease in discharge (Leake and others, 2010). Capture that results in a loss of water in streams, rivers, and wetlands now is a concern in many parts of the United States. Hydrologists commonly use analytical and numerical approaches to study temporal variations in sources of water to wells for select points of interest. Much can be learned about coupled surface/groundwater systems, however, by looking at the spatial distribution of theoretical capture for select times of interest. Development of maps of capture requires (1) a reasonably well-constructed transient or steady state model of an aquifer with head-dependent flow boundaries representing surface water features or evapotranspiration and (2) an automated procedure to run the model repeatedly and extract results, each time with a well in a different location. In this exercise, we will perform a streamflow capture analysis of the Freyberg model domain by developing a MODFLOW model, running it as many times as there are active model cells, and then creating a streamflow capture fraction map to summarize the results.

[Leake, S. A., Reeves, H. W. and Dickinson, J. E. (2010), A New Capture Fraction Method to Map How Pumpage Affects Surface Water Flow. Ground Water, 48: 690–700. doi: 10.1111/j.1745-6584.2010.00701.x](http://onlinelibrary.wiley.com/doi/10.1111/j.1745-6584.2010.00701.x/abstract)



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy

#### Load existing freyberg model

The MODFLOW 6 version of the freyberg model is located in:

```
../data/freyberg
```

The model name is `freyberg6`.

You should define the model workspace (`ws`) where the model files are, the model name (`name`), and the name and path of the model executable (`exe_name`). 

In [ ]:
ws = '../data/freyberg'
name = 'freyberg6'
exe_name = os.path.abspath('../bin/mf6')

# load simulation


#### Change the model workspace and run the model

The model workspace can be changed using `sim.set_sim_path(ws)`, where `ws` is set to be `data/freyberg`. Next write the simulation using `sim.write_simulation()` and run the model using `sim.run_simulation()`.

#### Extract the river results for the base model

Load the `RIV` results from `freyberg6.cbc` using `flopy.utils.CellBudgetFile(pth, precision='double')` and the `get_data(text=RIV)` method.

The river flux data is the `q` dtype in the river data. The data returned by `.get_data()` is a numpy recarray so the total stream flow can be calculated directly using `.q.sum()`.

#### Add additional wells and perform streamflow capture analysis

First get the gwf model object so that we can add a new well package to perturb the stream flow in each cell. You can get a list of the available models in the simulation using `sim.model_names`. Get the gwf model object using `sim.get_model()`.

In [ ]:
gwf = sim.get_model('freyberg6')

We will need the idomain and the CHD locations so that we only add wells in active cells. The idomain can be retrieved using `gwf.dis.idomain.array`. It will be useful to have the number of rows and columns in the model.

Determine cells with constant head boundary conditions (`cellid`) in `.stress_period_data.get_data()[0]`. 

In [ ]:
chd = gwf.get_package('CHD-1')
cellid = chd.stress_period_data.get_data()[0].cellid

Set idomain to zero in cells with constant heads

In [ ]:
for ipos in cellid:
    idomain[ipos] = 0

Make an array to store the values

In [ ]:
capture = np.zeros(idomain.shape, dtype=np.float)

#### Streamflow capture analysis code block
The code block below loops through every cell in the model and for each active cell adds a well in the current cell, rewrites the well file, reruns the model, extracts river leakage results from the model, and calculates the streamflow capture fraction for the cell. The model is run  with `silent=True` to suppress model output to the screen.

Streamflow capture is defined as 

$c_{k,i,j} = \frac{q_{k,i,j} - q_{{k,i,j}_{\text{base}}}}{|q_{\text{well}}|}$,

where $q_{\text{well}}$ is the pumping rate applied in each cell (use `-0.001`), $q_{k,i,j}$ is the net simulated river flux, and $q_{{k,i,j}_{\text{base}}}$ is the net simulated river flux from the base model. 

In [ ]:
wnam = gwf.name + '_cf.wel'
qwell = -1e-3
k = 0
for i in range(nrow):
    for j in range(ncol):
        # skip inactive cells
            
        # make a new well package
        
        # write the simulation files
        
        # run the simulation
        
        # process the results
        
        # add the value to the capture array
        
        # remove the new well package so it can be readded


Plot the capture fraction results using `flopy.plot.PlotMapView(model=gwf)` and `.plot_array()`.